In [1]:
import numpy as np
data = np.load('C:\\Users\\zihan\\Desktop\\Files\\Research\\bryankelly_without_missing.npz')
processed_data = data['processed_data']
T, N, M = processed_data.shape
returns = processed_data[:, :, 1]
# The dimension of returns: T*N
chara_data = processed_data[:, :, [8, 17, 33, 37]]

In [3]:
def get_tree(depth, res, value_array, L, d, chara_data, parent):
    if depth == d:
        final_array = np.ones([N, 1])
        for k in range(L):
            temp = value_array[:, k].reshape([N, 1])
            final_array = final_array * temp
        print(np.sum(final_array))
        if np.sum(final_array) < 10 ** (-3):
            print(final_array)
        final_array = 1.0 * final_array / np.sum(final_array)

        # if np.sum(final_array)<10**(-3):
        #     print (value_array)
        res.append([final_array])
    else:
        for i in range(L):
            for j in range(2):
                if len(parent) != 0:
                    index = np.ones((N), dtype=bool)
                    for k in parent:
                        index = (index) & ((value_array[:, k] != 0))
                    data_median = np.median(chara_data[i, index])
                else:
                    index = (value_array[:, i] != 0)
                    data_median = np.median(chara_data[i, index])
                temp_array = value_array.copy()
                parent.append(i)
                if j == 0:
                    temp_array[(index) & (chara_data[i, :] <= data_median), i] = 0
                    get_tree(depth + 1, res, temp_array, L, d, chara_data, parent)
                elif j == 1:
                    temp_array[(index) & (chara_data[i, :] >= data_median), i] = 0
                    get_tree(depth + 1, res, temp_array, L, d, chara_data, parent)
                parent.pop()
portfolio = np.zeros((T - 1, (2*4)**4))
for i in range(T-1):
    if i%20==0:
            print ('Have gone through %d' %(i))
    res = []
# The first parameter is number of characteristics, and the second is the depth of the tree.
    get_tree(0, res, np.ones([N, 4]), 4, 4, chara_data[i,:].T)
    # output = np.array(res[0][0])
    # output = np.reshape(output, [N, 1])
    # for j in range(1, len(res)):
    #     output = np.append(output, np.array(res[j][0]), axis=1)
    # portfolio[i, :] = returns[i + 1, :].dot(output)
    for j in range(len(res)):
        portfolio[i][j]= returns[i+1,:].dot(res[j])

Have gone through 0


Have gone through 20


Have gone through 40


Have gone through 60


Have gone through 80


...

Have gone through 260


Have gone through 280


Have gone through 300


Have gone through 320


Have gone through 340


In [8]:
gamma= 100
# portfolio_demean = portfolio - np.mean(portfolio, axis=0)
# Dimension of portfolio: T*M
covmat = portfolio.T.dot(portfolio) 
portfolio_mean= np.sum(portfolio, axis= 0, keepdims=True)
covmat= covmat+ portfolio_mean.T.dot(portfolio_mean)/(T-1)*gamma
variance, loading = np.linalg.eig(covmat)
output= []
for K in range(1, 20):
    loading_chosen = loading[:, :K]
    Factor = portfolio.dot(loading_chosen)
    mu = np.mean(Factor, axis=0)
    covmat_factor = np.cov(Factor.T)
    if K==1:
        output.append (abs(mu)*covmat_factor**(-1/2))
    else:
        output.append(np.sqrt(np.dot(np.dot(mu, np.linalg.inv(covmat_factor)), mu.T)))
twodecimals = ["&%.2f" % np.real(v) for v in output]
print (''.join(twodecimals))

&0.30&0.30&0.34&0.45&0.45&0.48&0.50&0.51&0.52&0.52&0.52&0.54&0.55&0.56&0.58&0.58&0.59&0.62&0.62
